In [ ]:
import sys

# runtype = "daily"
# report_date = "2020-04-30" #automated

no_of_cpu = 8
max_cores = 8
executor_mem = '56g'

Job_Name = "Devnandan :Activity10_Claim_Settlement"

print ("JOB NAME = "+str(Job_Name))
print ("No_of_cpu="+str(no_of_cpu))
print ("Max_cores="+str(max_cores))
print ("Executor_mem="+str(executor_mem))
# print ("Runtype="+str(runtype))
# print ("Report_date="+str(report_date))


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = 'gpanalytics'
es_pwd = ''
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'
spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################
conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
conf.set('spark.executor.memory',executor_mem)
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','120m')
conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('spark.executor.heartbeatInterval','120s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70");
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer");
conf.set("spark.broadcast.compress", "true");
conf.set("spark.shuffle.compress", "true");
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.driver.maxResultSize", "4g");
conf.set("spark.app.name", Job_Name)
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','12g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','300')
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)
conf.set("spark.scroll.keepalive","120m")

# conf.set('es.nodes',es_nodes)
# conf.set('es.port',es_port)
# conf.set('spark.es.net.http.auth.user', es_user)
# conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *
from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark import SparkContext, SparkConf
conf = pyspark.SparkConf()
#import numpy
import calendar
import time
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType, StringType, DecimalType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
import re # to use regular expressions

In [ ]:
from pyspark.sql.functions import hash
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

In [ ]:

prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

# Define the GPDB-Python connection options for PROD 
import psycopg2
import datetime
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)


In [ ]:

def load_gpdb_jdbc(col_str,dbtable,dbschema=dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .select(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .select(col_str).drop_duplicates()
   
    return data

In [ ]:
def get_latest_progress():
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress1",#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name')=='customer_activity')\
            .filter(col('source')=='activity10')
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(2020, 4, 30, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

In [ ]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        
                


        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                         time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
        

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

In [ ]:
# Define the Generic update function which takes the records currently in temp table (created with prefix 1 to original customercoe table)
# The function would first try to insert the records in original table. It that fails then it will upsert the records
# Since there is no direct upsert query an update and insert query with where clause is used
# This funciton just runs the SQL queries in GPDB and does not use spark
def update_activity(conn_to,table_name,source,gpdb_spark_options = prod_gpdb_spark_options,
                  schema="customermart"):
    conn_to.rollback()
    primary_key = ['source_system_name','source_system_customer_id','activity_type','activity_id','commit_timestamp']
    update_fields = ['feature_value_1', 'feature_value_2', 'feature_value_3', 'feature_string_1','feature_string_2','feature_string_3', 
                     'feature_string_4', 'feature_date_1', 'feature_date_2', 'feature_date_3']
    df_1 = sqlContext.read.format("greenplum").options(**gpdb_spark_options).option("dbschema",schema).option("dbtable","{}_10_staging".format(table_name)).load()
    out_columns = df_1.columns
    out_columns = [col for col in out_columns if "json" not in col]

    cur_to = conn_to.cursor()  

    only_insert_query = """insert into {schema}.{table_name} ({out_columns}) select {out_columns} 
    from {schema}.{table_name}_10_staging""".format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns))
    update_query = """UPDATE {schema}.{table_name} orig
                          SET
                            {update_fields}
                          FROM
                            {schema}.{table_name}_10_staging temp
                           WHERE 
                            {primary_key}
                       """.format(schema=schema,table_name=table_name,primary_key= " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]),update_fields = " , ".join([ "{key} = temp.{key}".format(key=key) for key in update_fields]))
    insert_query = """ INSERT INTO {schema}.{table_name} ({out_columns})
                           SELECT {out_columns}
                           FROM
                             {schema}.{table_name}_10_staging temp
                           WHERE
                             NOT EXISTS (
                             SELECT 1 FROM {schema}.{table_name} orig WHERE 
                            {primary_key}
                            )
                        """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns),primary_key = " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]))

    try:
        print("Inside try segment")
        cur_to.execute(only_insert_query)
        print("Executed Only insert query")
#         update_progress(table_name,source,time_filter,records)
    except Exception as e:
        print("Inside except segment")
        conn_to.rollback()
        cur_to.execute(update_query)
        print("Executed update query")
        cur_to.execute(insert_query)
        print("Executed insert query")
#         update_progress(table_name,source,time_filter,records)
    conn_to.commit()

In [ ]:
latest_progress=get_latest_progress()
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-2 #currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

In [ ]:
    
col_str=["claim_net", "claim_feature_concat", "type_of_claim", "claimant_name", "clm_create_date", "customer_id", "sum_insured_for_the_coverage_selected", "report_date", "num_claim_no","nett_cheque_amount", "relationship", "claim_status_cd", "dat_transaction_date", "cheque_date", "claim_no", "repudiated_reason"]

claim_report=load_gpdb_jdbc(col_str,"claim_report_gc_p2","datamarts","report_date",time_filter)\
.filter(col("claim_status_cd").isin('Close'))\
.filter(col("claim_feature_concat").isNotNull())\
.filter(col("customer_id").isNotNull())


In [ ]:
claim_report=claim_report.withColumn('activity_id', col('claim_feature_concat'))\
.withColumn('feature_value_3', col('nett_cheque_amount'))\
.withColumn('feature_date_1', col('dat_transaction_date'))\
.withColumn('feature_date_3', col('cheque_date'))\
.withColumn('feature_string_3', col('claimant_name'))\
.withColumn('commit_timestamp', col('report_date'))\
.withColumn('feature_string_1', col('relationship'))\
.withColumn('feature_string_2', col('claim_status_cd'))\
.withColumn('source_system_customer_id', col('customer_id'))


In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "claim_bpm_fraudmodelprediction",
                    "server.port": '1150-1170'}

bpm_fraud = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                                  .selectExpr("claimnumber", "risk_score")
bpm_fraud.columns

In [ ]:
bpm_fraud=bpm_fraud.withColumn('feature_value_1', col('risk_score'))

In [ ]:
    
col_str=["nps_category", "claim_number"]

nps_feedback=load_gpdb_jdbc(col_str,"bpm_nps_feedback","public")

In [ ]:
nps_feedback=nps_feedback.withColumn('feature_value_2', when(col('nps_category').isin('Passive'), lit(1))\
                                                .when(col('nps_category').isin("Detractor"), lit(2))\
                                                .when(col('nps_category').isin('Promoter'), lit(3)))
                                                       

In [ ]:
col_str=["claimnumber", "rpdreason"]

featuresummary=load_gpdb_jdbc(col_str,"claim_bpm_featuresummary","public")

In [ ]:
#claim_gc_gc_clm_payment_details
col_str=["date_of_approval", "num_claim_no"]

clm_payment=load_gpdb_jdbc(col_str,"claim_gc_gc_clm_payment_details","public")

In [ ]:
clm_payment=clm_payment.withColumn('num_claim_no_1', col('num_claim_no').cast('string'))

In [ ]:
clm_payment=clm_payment.withColumn('feature_date_2', col('date_of_approval'))

In [ ]:
df_1= claim_report.join(bpm_fraud, claim_report.claim_no == bpm_fraud.claimnumber, "left_outer")\
.drop(bpm_fraud.claimnumber)

In [ ]:
df_2= df_1.join(nps_feedback, df_1.claim_no == nps_feedback.claim_number, "left_outer")\
.drop(nps_feedback.claim_number)

In [ ]:
df_3= df_2.join(featuresummary, df_2.claim_no == featuresummary.claimnumber, "left_outer")\
.drop(featuresummary.claimnumber)

In [ ]:
df_main= df_3.join(clm_payment, df_3.claim_no == clm_payment.num_claim_no_1, "left_outer")\
.drop(clm_payment.num_claim_no)

In [ ]:
df_main=df_main.withColumn('source_system_name', lit('GC'))\
.withColumn('activity_type', lit('Claim Settlement'))\
.withColumn('feature_value_1',F.col('feature_value_1')*1.0)\
.withColumn('feature_value_2',F.col('feature_value_2')*1.0)

In [ ]:
df_main.columns

In [ ]:
from pyspark.sql.functions import coalesce,col,length,trim
df_main=df_main.withColumn('feature_string_4',when((length(trim(col('rpdreason'))) != 0)| (length(trim(col('repudiated_reason'))) != 0)\
                                              ,coalesce(df_main.rpdreason,df_main.repudiated_reason)).otherwise(lit(None)))

In [ ]:
df_main=df_main.select(col('source_system_name').cast('string'),col('source_system_customer_id').cast('string'),col('activity_type').cast('string'),col('activity_id').cast('string'),\
                         col('feature_value_1').cast(DoubleType()),col('feature_value_2').cast(DoubleType()),col('feature_value_3').cast(DoubleType()),\
                         col('feature_string_1').cast('string'),col('feature_string_2').cast('string'),col('feature_string_3').cast('string'),col('feature_string_4').cast('string'),\
                         col('feature_date_1').cast("timestamp"),col('feature_date_2').cast("timestamp"),col('feature_date_3').cast("timestamp"),col('commit_timestamp').cast("timestamp"))
                         

In [ ]:
df_main.columns

In [ ]:
# from pyspark.sql.window import Window

# df_main = df_main.withColumn("rn", F.row_number().over(Window.partitionBy('source_system_customer_id','activity_id'))
#         .orderBy(F.col("commit_timestamp").desc(),F.col("feature_value_3").desc()))

# df_main = df_main.filter(F.col("rn") == 1).drop("rn")

In [ ]:
## Updated the below window function (by adding : orderBY clause on feature_value_3)

from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1 = Window.partitionBy('source_system_customer_id','activity_id').orderBy(col("commit_timestamp").desc(),col("feature_value_3").desc())
df_main = df_main.withColumn('row_num',F.row_number().over(w1))
df_main = df_main.filter('row_num == 1')

In [ ]:
df_main = df_main.drop('row_num')

In [ ]:
df_main.persist(pyspark.StorageLevel.MEMORY_AND_DISK)

In [ ]:
records=df_main.count()
records

In [ ]:
output_index = "customer_activity_10_staging"
schema = "customermart"

import sys
try:
    
    df_main.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

In [ ]:
table_name='customer_activity'
source='activity10'
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod

In [ ]:
update_activity(conn_to,table_name,source,prod_gpdb_spark_options, schema="customermart")

In [ ]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbtable": "customermart.customer_activity",
         "server.port":"1150-1170"}


activity10_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.select('source_system_name', 'source_system_customer_id', 'activity_type', 'activity_id', 'feature_value_1', 'feature_value_2', 'feature_value_3', 'feature_string_1', 'feature_string_2', 'feature_string_3', 'feature_string_4', 'feature_date_1', 'feature_date_2', 'feature_date_3', 'commit_timestamp')\
.filter(col('activity_type')=='Claim Settlement')\
.filter(col('commit_timestamp').between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd'))).dropDuplicates()

In [ ]:
SrcGCIDCount=str(df_main.select(countDistinct("source_system_customer_id")).collect()[0][0])

In [ ]:
from pg import DB
db = DB(dbname="gpadmin", user='gpspark', passwd='spark@456', host='10.35.12.194', port= 5432)
GCidCount = db.query("select count(distinct source_system_customer_id) from customermart.customer_activity_10_staging").getresult()

In [ ]:
df_stag=df_main.select(['source_system_name','source_system_customer_id','activity_type','activity_id','commit_timestamp','feature_value_1', 'feature_value_2', 'feature_value_3', 'feature_string_1','feature_string_2','feature_string_3', 
                     'feature_string_4', 'feature_date_1', 'feature_date_2', 'feature_date_3'])

In [ ]:
activity10_df1=activity10_df.select(['source_system_name','source_system_customer_id','activity_type','activity_id','commit_timestamp','feature_value_1', 'feature_value_2', 'feature_value_3', 'feature_string_1','feature_string_2','feature_string_3', 
                     'feature_string_4', 'feature_date_1', 'feature_date_2', 'feature_date_3'])

In [ ]:
df_MissingData=df_stag.exceptAll(activity10_df1).withColumn('DataMisMatch',lit('Missing values at Target'))

In [ ]:
reconresult=[{
    "SrcCount":SrcGCIDCount,
    "TargetCount":GCidCount[0][0],
    "TrgtMissMatchCount":df_MissingData.count()
}]


In [ ]:
from pyspark.sql import Row
df_recon=sqlContext.createDataFrame(Row(**x) for x in reconresult)

In [ ]:
df_recon.show()

In [ ]:
df_recon=df_recon.withColumn('Status',when((df_recon.SrcCount==df_recon.TargetCount) & (df_recon.TrgtMissMatchCount==0),'Success').otherwise('Failed'))
status=df_recon.select(df_recon.Status).collect()[0][0]

In [ ]:
update_progress(table_name,source,time_filter,records,start_time,starttime,status)

In [ ]:
df_main.unpersist()

In [ ]:
spark.stop()